#**NEWS ANALYSIS**

---



##Example

In [1]:
# Install required libraries
!pip install newspaper3k transformers vaderSentiment pandas

# Importing necessary libraries
from newspaper import Article
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 4.7 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=81476140dc12a248fa79b447dfb531e10660fca6ce03dd7058309db8d9acd4bd
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=13c17b02e94c8b56a3a563d6b87ab408b40644ea203c18e19215dd500645b65f
  Stored in d

In [2]:
# Function to extract article content
def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

# Function to summarize the article
def summarize_article(article_text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(article_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    return summary

In [3]:
# List of article URLs
urls = [
    'https://www.cnbc.com/2024/09/19/stock-market-today-live-updates.html',
    'https://www.investopedia.com/dow-jones-today-09032024-8705310',
    'https://finance.yahoo.com/news/stock-market-today-sp-500-nasdaq-slip-as-sluggish-start-to-september-continues-200356910.html',
    'https://www.cnbc.com/2024/09/17/stock-market-today-live-updates.html',
    'https://www.cnbc.com/2024/09/15/stock-futures-are-little-changed-as-investors-await-major-fed-decision-live-updates.html'
]

###Sentiment Intensity Analyzer

In [4]:
# Function to perform sentiment analysis
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)
    if sentiment_scores['compound'] >= 0.05:
        sentiment_label = 'Positive'
    elif sentiment_scores['compound'] <= -0.05:
        sentiment_label = 'Negative'
    else:
        sentiment_label = 'Neutral'
    return sentiment_label, sentiment_scores  # Return both the label and the scores


In [5]:
# List to store results
results = []

# Loop through the URLs
for url in urls:
    try:
        # Extract article content
        article_text = extract_article(url)

        # Summarize the article
        summary = summarize_article(article_text)

        # Perform sentiment analysis on the summary
        sentiment_label, sentiment_scores = analyze_sentiment(summary)

        # Append the result (adding sentiment scores)
        results.append({
            "URL": url,
            "Summary": summary,
            "Sentiment": sentiment_label,
            "Compound": sentiment_scores['compound'],
            "Positive": sentiment_scores['pos'],
            "Neutral": sentiment_scores['neu'],
            "Negative": sentiment_scores['neg']
        })

    except Exception as e:
        print(f"Error processing {url}: {str(e)}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,URL,Summary,Sentiment,Compound,Positive,Neutral,Negative
0,https://www.cnbc.com/2024/09/19/stock-market-t...,"The 30-stock Dow inched up 38.17 points, or 0....",Positive,0.2500,0.040,0.960,0.000
1,https://www.investopedia.com/dow-jones-today-0...,Major U.S. stock indexes closed sharply lower ...,Positive,0.8074,0.145,0.822,0.033
2,https://finance.yahoo.com/news/stock-market-to...,"US stocks were mixed on Wednesday, seesawing t...",Negative,-0.0516,0.056,0.884,0.060
3,https://www.cnbc.com/2024/09/17/stock-market-t...,The Dow Jones Industrial Average slid 103.08 p...,Negative,-0.3182,0.000,0.931,0.069
4,https://www.cnbc.com/2024/09/15/stock-futures-...,"The S&P 500 added 0.13%, ending at 5,633.09. T...",Positive,0.1027,0.070,0.874,0.056


In [6]:
# Save DataFrame to an Excel file
file_name = 'sentiment_analyzer.xlsx'
df.to_excel(file_name, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###TextBlob

In [7]:
!pip install textblob
from textblob import TextBlob

In [8]:
# Function to perform sentiment analysis with TextBlob
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity  # Polarity score
    subjectivity = blob.sentiment.subjectivity  # Subjectivity score
    # Determine sentiment label based on polarity
    if polarity > 0:
        sentiment_label = 'Positive'
    elif polarity < 0:
        sentiment_label = 'Negative'
    else:
        sentiment_label = 'Neutral'
    return sentiment_label, polarity, subjectivity

In [9]:
# List to store results
results = []

# Loop through the URLs
for url in urls:
    try:
        # Extract article content
        article_text = extract_article(url)

        # Summarize the article
        summary = summarize_article(article_text)

        # Perform sentiment analysis on the summary
        sentiment_label, polarity, subjectivity = analyze_sentiment(summary)

        # Append the result (including polarity and subjectivity scores)
        results.append({
            "URL": url,
            "Summary": summary,
            "Sentiment": sentiment_label,
            "Polarity": polarity,
            "Subjectivity": subjectivity
        })

    except Exception as e:
        print(f"Error processing {url}: {str(e)}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
df

,URL,Summary,Sentiment,Polarity,Subjectivity
0,https://www.cnbc.com/2024/09/19/stock-market-t...,"The 30-stock Dow inched up 38.17 points, or 0....",Positive,0.189273,0.418909
1,https://www.investopedia.com/dow-jones-today-0...,Major U.S. stock indexes closed sharply lower ...,Negative,-0.036265,0.379321
2,https://finance.yahoo.com/news/stock-market-to...,"US stocks were mixed on Wednesday, seesawing t...",Positive,0.040000,0.290000
3,https://www.cnbc.com/2024/09/17/stock-market-t...,The Dow Jones Industrial Average slid 103.08 p...,Negative,-0.125000,0.250000
4,https://www.cnbc.com/2024/09/15/stock-futures-...,"The S&P 500 added 0.13%, ending at 5,633.09. T...",Positive,0.400000,0.600000


In [11]:
# Save DataFrame to an Excel file
file_name = 'sentiment_textblob.xlsx'
df.to_excel(file_name, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#END

---